# Balance de energía

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate
import import_ipynb

In [ ]:
from calculos_Q_invierno import Q_oficinas, dot_Q_oficinas, T_oficinas, T_exterior_inv
Q_oficinas_invierno = Q_oficinas
dot_Q_oficinas_invierno = dot_Q_oficinas
T_oficinas_invierno = T_oficinas
T_exterior_invierno = T_exterior_inv

from calculos_Q_verano import Q_oficinas, dot_Q_oficinas, T_oficinas, T_exterior_ver
Q_oficinas_verano = Q_oficinas
dot_Q_oficinas_verano = dot_Q_oficinas
T_oficinas_verano = T_oficinas
T_exterior_verano = T_exterior_ver

**Balance de masa**

$$
\dot{m}W_{exterior} + \dot{m}_{gen} = \dot{m}W_{interior}
$$

**Balance de energía**

$$
\dot{Q}_{in} + \dot{Q}_{gen} + \dot{m}h_{exterior} = \dot{Q}_{loss} + \dot{m}h_{interior}
$$

In [1]:
# Cálculo de Q_gen
mets = 1.5 * 58.2
n_personas = 1.5
peso = 84.9
altura = 1.75
area_superficial = 0.202 * peso**0.425 * altura**0.725
Q_gen_personas = n_personas * mets * area_superficial
print(f"Energía generada por las personas en la oficina: {Q_gen_personas:.2f} W")

Energía generada por las personas en la oficina: 262.09 W


Fórmulas de psicrometría

In [ ]:
def humedad_absoluta(T, RH):
    """Calcula la humedad absoluta (kg/m3) a partir de la temperatura (°C) y la humedad relativa (%)."""
    # Presión de saturación del agua (Pa)
    P_sat = 610.78 * np.exp((17.27 * T) / (T + 237.3))
    # Presión parcial del vapor de agua (Pa)
    P_vapor = RH / 100 * P_sat
    # Humedad absoluta (kg/m3)
    humedad_abs = 0.622 * P_vapor / (101325 - P_vapor) * 1000  # Convertir a g/m3
    return humedad_abs

def entalpia_aire(T, RH):
    """Calcula la entalpía del aire (kJ/kg) a partir de la temperatura (°C) y la humedad relativa (%)."""
    humedad_abs = humedad_absoluta(T, RH) / 1000  # Convertir a kg/m3
    h = 1.006 * T + humedad_abs * (2501 + 1.86 * T)  # kJ/kg
    return h

m_gen = n_personas * 0.1  # kg/s, asumiendo 0.1 kg/s por persona CAMBIAR

## Cálculos para el invierno

Resolver sistema de ecuaciones para determinar $\dot{Q}_{in}$ y $\dot{m}$

In [ ]:
W_interior = entalpia_aire(T_oficinas, 50)  # kJ/kg, asumiendo 50% RH interior
W_exterior_inv = entalpia_aire(T_exterior_inv, 80)  # kJ/kg, asumiendo 80% RH exterior invierno

m = m_gen / (W_exterior_inv - W_interior)  # kg/s
print(f"Caudal de aire necesario para remover la humedad: {m:.2f} kg/s")

h_exterior_inv = entalpia_aire(T_exterior_inv, 80)  # kJ/kg
h_interior = entalpia_aire(T_oficinas, 50)  # kJ/kg

for oficina, perdidas in Q_oficinas_invierno.items():
    Q_in = perdidas + m * (h_interior - h_exterior_inv) - Q_gen_personas  # W
    print(f"Potencia necesaria para calefacción en {oficina} durante el invierno: {Q_in:.2f} W")

## Cálculos para el verano

In [ ]:
W_interior = entalpia_aire(T_oficinas, 50)  # kJ/kg, asumiendo 50% RH interior
W_exterior_ver = entalpia_aire(T_exterior_ver, 80)  # kJ/kg, asumiendo 80% RH exterior invierno

m = m_gen / (W_exterior_ver - W_interior)  # kg/s
print(f"Caudal de aire necesario para remover la humedad: {m:.2f} kg/s")

h_exterior_ver = entalpia_aire(T_exterior_ver, 80)  # kJ/kg
h_interior = entalpia_aire(T_oficinas, 50)  # kJ/kg

for oficina, perdidas in Q_oficinas_verano.items():
    Q_in = perdidas + m * (h_interior - h_exterior_ver) - Q_gen_personas  # W
    print(f"Potencia necesaria para climatización en {oficina} durante el verano: {Q_in:.2f} W")